<a href="https://colab.research.google.com/github/PacktPublishing/Modern-Computer-Vision-with-PyTorch-2E/blob/main/Chapter18/convert_to_onnx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
try:
  from torch_snippets import *
except:
  %pip install torch-snippets gitPython lovely-tensors
  from torch_snippets import *

from git import Repo

repository_url = 'https://github.com/sizhky/quantization'
destination_directory = '/content/quantization'
if exists(destination_directory):
  repo = Repo(destination_directory)
else:
  repo = Repo.clone_from(repository_url, destination_directory)

%cd {destination_directory}
%pip install -qq -r requirements.txt # this will take about 5 min of time
%pip install onnxruntime-gpu onnx
# print(repo.git.pull('origin', 'main'))

# Train

In [ ]:
# Change to `Debug=false` in the line below
# to train on a larger dataset
%env DEBUG=true
!make train

env: DEBUG=true
python -m src.defect_classification.train
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100% 528M/528M [00:07<00:00, 78.3MB/s]
Generating train split: 100% 2331/2331 [00:02<00:00, 1049.98 examples/s]
Generating valid split: 100% 1004/1004 [00:01<00:00, 884.39 examples/s]
Class Balance


# Benchmarks

In [ ]:
sys.path.append('src')
from defect_classification.model import SDD
from defect_classification.train import process_example, DefectsDataset
from datasets import load_dataset

In [ ]:
val_ds = load_dataset('sizhkhy/kolektor_sdd2', split="valid[:50]+valid[-50:]")
val_ds = val_ds.map(process_example).remove_columns(['split', 'path'])
val_ds.set_format("pt", columns=["image", "label"], output_all_columns=True)
val_ds = DefectsDataset(val_ds)
val_dl = DataLoader(val_ds, batch_size=32, shuffle=True, drop_last=True)

In [ ]:
device = 'cpu'
model = torch.load('model.pth').to(device)

In [ ]:
# prompt: export to onnx with dynamic axes
model.eval()
i, _ = next(iter(val_dl))
with torch.no_grad():
    # first prediction is model warmup
    model(i.to(device))
    print(f'Time taken by pytorch model on sample input')
    %time pred_pytorch_model = model(i.to(device))
    pred_pytorch_model = pred_pytorch_model.to(device).numpy().reshape(-1)


Time taken by pytorch model on sample input
CPU times: user 15.2 s, sys: 2.81 s, total: 18 s
Wall time: 18 s


In [ ]:
input_names = ['image']
output_names = ['label']
dynamic_axes = {'image': {0: 'batch_size'}, 'label': {0: 'batch_size'}}
onnx_file_path = 'sdd_base.onnx'
with open(onnx_file_path, 'wb') as f:
    torch.onnx.export(
        model,
        i[:1].to(device),
        f,
        export_params=True,
        verbose=False,
        opset_version=13,
        do_constant_folding=True,
        input_names=input_names,
        output_names=output_names,
        dynamic_axes=dynamic_axes
    )


In [ ]:
from onnxruntime import InferenceSession
# load the onnx model on gpu
session = InferenceSession('sdd_base.onnx', providers=['CPUExecutionProvider'])
# make sample prediction
input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name

input = i.numpy()

# first prediction is model warmup
pred_onnx = session.run(None, {input_name: input})[0]
print(f'Time taken by ONNX model on same input')
%time pred_onnx = session.run(None, {input_name: input})
pred____onnx_model = pred_onnx[0].reshape(-1)

Time taken by ONNX model on same input
CPU times: user 13.7 s, sys: 468 ms, total: 14.1 s
Wall time: 15.3 s


In [ ]:
print('Both the pytorch and onnx model\'s predictions are identical - ')
np.allclose(
    pred_pytorch_model,
    pred____onnx_model,
)

Both the pytorch and onnx model's predictions are identical - 


True